In [2]:
!pip install pandas sklearn gym

In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from gym import spaces, Env
import numpy as np

In [4]:
class LineAgent:
    def __init__(self):
        self.vectors = np.zeros(shape=(2, 10))

    def predict(self, actions):
        self.vectors = self.vectors + actions


In [183]:
import math
from operator import truediv

from numpy import NaN, nan


class VectorEnv(Env):
    def __init__(self, frames):
        super(VectorEnv, self).__init__()
        self.action_space = spaces.Discrete(4,)
        self.nb_actions = 4
        self.observation_space = spaces.Box(shape=(150,), high=100, low=-100)
        self.frames = frames
        self.reset()

    def step(self, action):
        self.t += 1
        done = self.t > 100
        self.episode += 1
        self.episode = self.episode % 10
        if action[0] == 0:
            self.vectors[self.episode] = self.vectors[self.episode][0] + 1
        if action[0] == 1:
            self.vectors[self.episode] = self.vectors[self.episode][0] - 1
        if action[0] == 2:
            self.vectors[self.episode] = self.vectors[self.episode][1] + 1
        if action[0] == 3:
            self.vectors[self.episode] = self.vectors[self.episode][1] - 1

        # Calculate reward based on vectors and frame
        reward = 0
        vectors_with_start = np.concatenate(([self.start], self.vectors[1:]))
        vectors_with_end = np.concatenate((self.vectors[:-1], [self.end]))
        for (v1, v2) in zip(vectors_with_start, vectors_with_end):
            for box in self.boxes:
                reward -= -1000 if self.intersects(v1, v2, box) else 0

        # Calculate distance of linesegments and reward for shorter lines

        return self.frame, reward, done, False, {}

    def reset(self):
        self.vectors = np.zeros(shape=(10, 2))
        i = np.random.randint(len(self.frames), size=1)
        self.frame = self.frames[i]
        print("gg", self.frame.shape, i)
        self.boxes = self.chunks(self.frame.flatten(), 4)
        self.t = 0
        self.episode = 0
        self.start = self.boxes[0][:2]
        self.end = self.boxes[-1][:2]
        self.state = self.frame
        return self.frame

    def intersects(self, v1, v2, box):
        x1 = v1[0]
        y1 = v1[1]
        x2 = v2[0]
        y2 = v2[1]
        bx = box[0]
        by = box[1]
        bw = box[2]
        bh = box[3]
        box_lines = [ (bx, by, bx, by + bh), (bx, by - bh, bx + bw, by - bh), (bx + bw, by - bh, bx - bw, by), (bx, by, bx + bh, by) ]
        result = False
        for (bx_, by_, bx2_, by2_) in box_lines:
            if result:
                return result
            else:
                result = self.ccw(x1, y1, bx_, by_, bx2_, by2_) != self.ccw(x2, y2, bx_, by_, bx2_, by2_) and self.ccw(x1, y1, x2, y2, bx_, by_) != self.ccw(x1, y1, x2, y2, bx2_, by2_)
        
        return result

    def chunks(self, lst, n):
        result = []
        for i in range(0, len(lst), n):
            if math.isnan(lst[i]):
                break
            else:
                result.append(lst[i:i + n])
        return result

    def ccw(self, x1, y1, x2, y2, x3, y3):
        return (y3-y1) * (x2-x1) > (y2-y1) * (x3-x1)

In [184]:
dataset = pd.read_csv(os.path.join(os.getcwd(), "../../data/vectors/vector.csv"), header = None, delimiter=',', names = list(range(150)))

In [185]:
env = VectorEnv(dataset.values)
print(env.action_space)
print(env.observation_space.shape)

obj, r, d, t, x = env.step(np.arange(0, 4, dtype=int).reshape(-1, 1))

gg (1, 150) [855]
Discrete(4)
(150,)


In [186]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape
from keras.optimizers import Adam

In [187]:
model = Sequential()
model.add(Dense(150, input_dim=150))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(4, activation='linear'))
model.add(Flatten())
print(model.input_spec)

[InputSpec(shape=(None, 150), ndim=2)]


In [188]:

!pip install keras-rl2
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn_only_embedding = DQNAgent(model=model, nb_actions=4, memory=memory, nb_steps_warmup=500, target_model_update=1e-2, policy=policy)
dqn_only_embedding.compile(Adam(lr=1e-3), metrics=['mae'])
dqn_only_embedding.fit(env, nb_steps=1000000, visualize=False, verbose=1, nb_max_episode_steps=99, log_interval=100000)

c:\Users\chwo\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 1000000 steps ...
gg (1, 150) [3835]
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected dense_82_input to have 2 dimensions, but got array with shape (1, 1, 1, 150)